In [1]:
import math, random, string

In [2]:
################################################################################
# Part 0: Utility Functions
################################################################################

COUNTRY_CODES = ['af', 'cn', 'de', 'fi', 'fr', 'in', 'ir', 'pk', 'za']

def start_pad(c):
    ''' Returns a padding string of length n to append to the front of text
        as a pre-processing step to building n-grams '''
    return '~' * c

def ngrams(c, text):
    ''' Returns the ngrams of the text as tuples where the first element is
        the length-n context and the second is the character '''
    
    # sliding window
    ng = []
    for x in range(len(text)):
        char = text[x]
        if x < c:
            context = start_pad(c - x) + text[:x] # x = 0, 0 chars, x = 1, 1 char, etc
        else:
            context = text[x-c:x]

        ng.append((context, char))
    return ng

def create_ngram_model(model_class, path, c=2, k=0):
    ''' Creates and returns a new n-gram model '''
    model = model_class(c, k)
    with open(path, encoding='utf-8', errors='ignore') as f:
        model.update(f.read())
    return model

In [3]:
ngrams(3, 'abcabc')

[('~~~', 'a'),
 ('~~a', 'b'),
 ('~ab', 'c'),
 ('abc', 'a'),
 ('bca', 'b'),
 ('cab', 'c')]

In [4]:
################################################################################
# Part 1: Basic N-Gram Model
################################################################################

class NgramModel(object):
    ''' A basic n-gram model using add-k smoothing '''

    def __init__(self, c, k):
        self.ngram_max = c
        self.smooth_par = k
        self.vocab = set()
        self.wc = {}

    def get_vocab(self):
        ''' Returns the set of characters in the vocab '''
        return self.vocab

    def update(self, text):
        ''' Updates the model n-grams based on text '''
        unique = set(text)
        self.vocab = self.vocab | unique # | = union
        wc = self.wc
        for g in ngrams(self.ngram_max, text):
            if g[0] in wc:
                if g[1] in wc[g[0]]:
                    wc[g[0]][g[1]] += 1
                else:
                    wc[g[0]][g[1]] = 1
            else:
                wc[g[0]] = {}
                wc[g[0]][g[1]] = 1
        
        self.wc = wc

    def prob(self, context, char):
        ''' Returns the probability of char appearing after context '''
        if context not in self.wc: # return 1/len(V) for a novel context
            return 1/len(self.vocab)
        
        k = self.smooth_par
        v = len(self.vocab)
        p = 0
        counts = self.wc[context]
        if char not in counts:
            c = 0
        else:
            c = counts[char]
            
        # p(char | context) = counts(context, char)/counts(context)
        p = (c + k) / (sum(counts.values()) + (k * v))

        return p

    def random_char(self, context):
        ''' Returns a random character based on the given context and the 
            n-grams learned by this model '''
        
        r = random.random()
        if context not in self.wc:
            x = r / (1/len(self.vocab)) # equal chance with no context
            x = math.floor(x)
            return list(self.vocab)[x]
        
        counts = self.wc[context]
        total =  sum(counts.values())
        curr = 0

        for k,v in counts.items():
            curr += (v / total)
            if r <= curr:
                return k

        return '-'

    def random_text(self, length):
        ''' Returns text of the specified character length based on the
            n-grams learned by this model '''
        
        c = self.ngram_max
        text = ""

        for i in range(length):
            if i >= c:
                context = text[(i - c):i]
            else:
                context = start_pad(c - i) + text[:i]

            text += self.random_char(context)

        return text

    def perplexity(self, text):
        ''' Returns the perplexity of text based on the n-grams learned by
            this model '''
        perp = 0
        c = self.ngram_max

        for w in range(len(text)): # take the geometric mean of all chars
            if w >= c:
                context = text[(w - c):w]
            else:
                context = start_pad(c - w) + text[:w]
            char = text[w]
            prob = self.prob(context, char)
            if not prob: # catch log(0) <- undefined
                return (float('inf'))

            perp -= math.log(prob) #
        
        perp = perp / (len(text))
        return math.e**(perp)

\begin{align*}
\log\left(Perplexity(W)\right) &= \log\left[\left(\prod_{i = 1}^{N} \frac{1}{P(w_i \mid w_1,\dots,w_{i-1})} \right)^{1/N}\right] \\
                               &= \frac{1}{N}\log\left[\prod_{i = 1}^{N} \frac{1}{P(w_i \mid w_1,\dots,w_{i-1})}\right] \\
                               &= \frac{1}{N} \sum_{i = 1}^{N} \log\left[\frac{1}{P(w_i \mid w_1,\dots,w_{i-1})}\right] \\
                               &= \frac{1}{N} \sum_{i = 1}^{N} \left[\log\left(1\right) - \log\left( P(w_i \mid w_1,\dots,w_{i-1} ) \right)\right] \\
                               &= \frac{1}{N} \sum_{i = 1}^{N} 0 - \log\left( P(w_i \mid w_1,\dots,w_{i-1} ) \right)\\
\log\left(Perplexity(W)\right) &= -\frac{1}{N} \sum_{i = 1}^{N} \log\left( P(w_i \mid w_1,\dots,w_{i-1} ) \right) 


\end{align*}

In [5]:
temp = NgramModel(1, 0)
temp.update('abab')
print(temp.get_vocab())
temp.update('abcd')
print(temp.get_vocab())

print(temp.prob('a','b'))
print(temp.prob('~','c'))
print(temp.prob('b','c'))

{'a', 'b'}
{'d', 'c', 'a', 'b'}
1.0
0.0
0.5


In [6]:
temp = NgramModel(0, 0)
temp.update('abab')
temp.update('abcd')
random.seed(1)
[temp.random_char('') for i in range(25)]

['a',
 'c',
 'c',
 'a',
 'b',
 'b',
 'b',
 'c',
 'a',
 'a',
 'c',
 'b',
 'c',
 'a',
 'b',
 'b',
 'a',
 'd',
 'd',
 'a',
 'a',
 'b',
 'd',
 'b',
 'a']

In [7]:
m = NgramModel(1, 0)
m.update('abab')
m.update('abcd')
random.seed(1)
m.random_text(25)

'abcdcdababcdbabcdbcddabab'

In [8]:
m = create_ngram_model(NgramModel, 'data/shakespeare_input.txt',2)
print(m.random_text(250))

Firt be don, bequardoseent. Lore himpon his him, Lornextee imparrociss,
And lin courepat inche
To to er somentrund arthenceight mestatch fich ourd: lothearguand lade this gracteare Dukeetche ill, way clord:
POLIA:
GRANTONSON:
It th bar le joingueelto


In [9]:
m = create_ngram_model(NgramModel, 'data/shakespeare_input.txt',3)
print(m.random_text(250))

First Cupill I
est-il?

Like Henription; the good from honough much bot: whath
where dearls to senance do,
Come stribestrust dust her ple goes itself me like no:
Sol
To wer
work, I'll comes a vilst Served:
O, be arthis.
So find with my fore nake; leg


In [10]:
m = create_ngram_model(NgramModel, 'data/shakespeare_input.txt',4)
print(m.random_text(250))

First Citizens:
Why, waited.

VALENTIO:
What you thine have no far and didst to siness Androus as conventions in me: a barbariation, sways
Drag pear it, and wealthou die, lord Pompey.

MISTRESS FORD:
Greath, I darest with his return, seed most the st


In [11]:
m = create_ngram_model(NgramModel, 'data/shakespeare_input.txt',7)
print(m.random_text(250))

First Citizen:
Ye're honest;
but yet drunk in hand: I am too choleric.

ANTIPHOLUS OF EPHESUS:
I needs as valiant youth will cut off, and be that, my lord,
As if this busy time.

CORIOLANUS:
Not I, sir; let me ha't: I have deposed.

WARWICK:
When we 


In [12]:
temp = NgramModel(1, 0)
temp.update('abab')
temp.update('abcd')
print(temp.perplexity('abcd'))
print(temp.perplexity('abca'))
print(temp.perplexity('abcda'))

1.189207115002721
inf
1.515716566510398


In [13]:
m = NgramModel(1, 0)
m.update('abab')
m.update('abcd')
print(m.prob('a', 'a'))
print(m.prob('a', 'b'))
print(m.prob('c', 'd'))
print(m.prob('d', 'a'))

0.0
1.0
1.0
0.25


In [14]:
################################################################################
# Part 2: N-Gram Model with Interpolation
################################################################################

class NgramModelWithInterpolation(NgramModel):
    ''' An n-gram model with interpolation '''

    def __init__(self, c, k):
        self.ngram_max = c
        self.smooth_par = k
        self.weights = [1/(c+1)] * (c+1)
        self.vocab = set()
        self.wc = {}

    def set_lambdas(self, lambdas):
        if len(lambdas) != (self.ngram_max + 1):
            print("Error: len(lambdas) does not match max ngram")
        s = sum(lambdas)
        if s < 1:
            print("Error: Weights do not add up to 1.")
        elif s > 1:
           print("sum(lambdas) > 1, normalizing by sum(lambdas)...\n")
           t = [x / s for x in lambdas]
        else:
            t = lambdas
        self.weights = t
        print(f"lambdas = {t}")

    def get_vocab(self):
        return self.vocab

    def update(self, text):
        unique = set(text)
        self.vocab = self.vocab | unique # | = union
        wc = self.wc
        for c in range(self.ngram_max,-1,-1):
            for g in ngrams(c, text):
                if g[0] in wc:
                    if g[1] in wc[g[0]]:
                        wc[g[0]][g[1]] += 1
                    else:
                        wc[g[0]][g[1]] = 1
                else:
                    wc[g[0]] = {}
                    wc[g[0]][g[1]] = 1
            
        self.wc = wc

    def prob(self, context, char):
        p_interp = 0
        lambdas = self.weights

        for l in range(len(lambdas)):
            t_context = context[l:]
            prob = super().prob(t_context, char) # use parent prob 
            p_interp += lambdas[l] * prob

        return p_interp

In [15]:
m = NgramModelWithInterpolation(1, 0)
m.update('abab')

print(m.prob('a', 'a'))
print(m.prob('a', 'b'))

0.25
0.75


In [16]:
m.set_lambdas([3,2])

sum(lambdas) > 1, normalizing by sum(lambdas)...

lambdas = [0.6, 0.4]


In [17]:
print(m.prob('a', 'a'))
print(m.prob('a', 'b'))

0.2
0.8


In [18]:
m = NgramModelWithInterpolation(2, 1)
m.update('abab')
m.update('abcd')

print(m.prob('~a', 'b'))
print(m.prob('ba', 'b'))
print(m.prob('~c', 'd'))
print(m.prob('bc', 'd'))

0.4682539682539682
0.4349206349206349
0.27222222222222225
0.3222222222222222


## Analysis

In [19]:
def get_model_scores(model_class, c = 2, k = 0, lambdas = []):
    model = create_ngram_model(model_class, 'data/shakespeare_input.txt', c = c, k = k)
    scores = {
        'nytimes' : [],
        'shakespeare' : []
    }

    if len(lambdas) > 0:
        model.set_lambdas(lambdas)

    with open('data/nytimes_article.txt', encoding='utf-8', errors = 'ignore') as file:
        for f in file:
            p = model.perplexity(f)
            if f == '\n':
                continue
            scores['nytimes'].append(p)

    with open('data/shakespeare_sonnets.txt', encoding='utf-8', errors = 'ignore') as file:
        for f in file:
            p = model.perplexity(f)
            if f == '\n':
                continue
            scores['shakespeare'].append(p)
    
    return scores

---

pandas is imported for analytics, not for the model.

In [32]:
for x in [0,1,2,5,20]:
    s = get_model_scores(NgramModel, c = 2, k = x)
    for k,v in s.items():
        print(f"{k}")
        print(f"{v}")
        print(f"E[{k}] = {sum(v)/len(v)}")
    print('')

nytimes
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
E[nytimes] = inf
shakespeare
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, i

In [33]:
for x in [0,1,2,5,20]:
    s = get_model_scores(NgramModel, c = 4, k = x)
    for k,v in s.items():
        print(f"{k}")
        print(f"{v}")
        print(f"E[{k}] = {sum(v)/len(v)}")
    print('')

nytimes
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
E[nytimes] = inf
shakespeare
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, i

In [38]:
for x in [[1,1,1],[10,5,1],[1,5,10],[10,1,10],[1,10,1]]:
    s = get_model_scores(NgramModelWithInterpolation, c = 2, k = 1, lambdas = x)
    for k,v in s.items():
        print(f"{k} = {v}")
        print(f"E[{k}] = {round(sum(v)/len(v),4)}")

sum(lambdas) > 1, normalizing by sum(lambdas)...

lambdas = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
nytimes = [12.362261354114844, 11.673487909117966, 11.251566586046698, 12.748407042101778, 14.449679338111618, 12.70633012697608, 17.143334580706878, 18.173602598954606, 18.821555143617196, 12.52692197492397, 12.752089665009747, 12.345359727512818, 16.360378059538505, 14.964151221407421, 11.421918162241399, 13.403279031897425, 11.506221713033696, 11.167267545886112, 12.776568919048486, 14.370323856849817]
E[nytimes] = 13.6462
shakespeare = [60.78427985301258, 11.473651736632036, 10.691141422876873, 9.926092489993602, 10.021406813893813, 9.37538091349768, 13.49440578187743, 11.240978345457401, 10.83640306495767, 9.746851662965366, 10.36313226619236, 10.04400836461502, 12.362227484654612, 11.941575378835028, 8.481103474283882, 72.71332710204099, 10.294475819282463, 12.164512417053292, 11.379780756273997, 10.726791914449445, 10.0652748898574, 9.68535974178551, 10.621272

In [39]:
for x in [[1,1,1],[10,5,1],[1,5,10],[10,1,10],[1,10,1]]:
    s = get_model_scores(NgramModelWithInterpolation, c = 2, k = 5, lambdas = x)
    for k,v in s.items():
        print(f"{k} = {v}")
        print(f"E[{k}] = {round(sum(v)/len(v),4)}")

sum(lambdas) > 1, normalizing by sum(lambdas)...

lambdas = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
nytimes = [12.5377099947066, 11.853422322223267, 11.42225426563974, 12.820717368956894, 14.624706090934273, 12.739030316589865, 16.781863956293854, 17.998170987274886, 18.567411290028907, 12.66821312211553, 12.990008100717645, 12.439188705084078, 16.252857274786205, 14.815143367275217, 11.491727210026612, 13.26803151427745, 11.601049490264415, 11.257868402995367, 12.84712633032034, 14.60269196812961]
E[nytimes] = 13.679
shakespeare = [60.502891001296426, 11.739652923619182, 10.850452182039302, 10.0220612438038, 10.180100520549797, 9.494382091046456, 14.026874808474384, 11.358650918945237, 10.983333522459475, 9.873908246641566, 10.531358563247654, 10.132340687034525, 12.572873367631669, 11.997194923618915, 8.680933711162211, 70.55847964053604, 10.39248241417502, 12.304369818727945, 11.616027937403107, 10.880000793580926, 10.149410433616541, 9.785248123601125, 10.73700

In [42]:
for x in [[1,1,1,1,1],[20,10,5,2,1],[1,2,5,10,20],[20,5,2,1,1],[10,5,1,1,1]]:
    s = get_model_scores(NgramModelWithInterpolation, c = 4, k = 1, lambdas = x)
    for k,v in s.items():
        print(f"{k}")
        print(f"{v}")
        print(f"E[{k}] = {round(sum(v)/len(v),4)}")
        print("")

sum(lambdas) > 1, normalizing by sum(lambdas)...

lambdas = [0.2, 0.2, 0.2, 0.2, 0.2]
nytimes
[9.495981742911452, 9.581472941370647, 8.862373750709487, 9.523792813816955, 11.688145867779175, 9.49386568406211, 12.78212517760296, 14.290477106001408, 15.768952293057284, 9.677966396150325, 9.862086391122885, 9.126472130583743, 13.250716772484873, 11.591428788941387, 9.342834547943577, 10.350619119401122, 8.793491475272747, 8.306795062437839, 10.156365696472674, 11.679492573634116]
E[nytimes] = 10.6813

shakespeare
[62.941900971078965, 8.094593198209438, 7.5464533024390965, 7.178222709176415, 7.633868990115561, 6.271260785346882, 10.221053856349869, 8.687528251789388, 8.060711555407076, 6.870745244130571, 8.414087699207395, 7.472704946286449, 9.72198416787534, 8.646305917299058, 6.491768701217365, 70.33869368369926, 7.652784148905666, 8.389449944037448, 8.75814782744257, 8.8587436749068, 7.1070148830942035, 6.9677812327344935, 7.940336258219983, 8.384690630894958, 8.13141431343051, 7.088832

In [43]:
for x in [[1,1,1,1,1],[20,10,5,2,1],[1,2,5,10,20],[20,5,2,1,1],[10,5,1,1,1]]:
    s = get_model_scores(NgramModelWithInterpolation, c = 4, k = 5, lambdas = x)
    for k,v in s.items():
        print(f"{k}")
        print(f"{v}")
        print(f"E[{k}] = {round(sum(v)/len(v),4)}")
        print("")

sum(lambdas) > 1, normalizing by sum(lambdas)...

lambdas = [0.2, 0.2, 0.2, 0.2, 0.2]
nytimes
[11.03620558360094, 10.76963338089487, 10.186265224550267, 10.64674074920788, 13.012645903187972, 10.804481832700532, 13.8443685797375, 15.62447170711393, 17.13617201745772, 10.841206703520998, 11.557252331405508, 10.35788584112795, 14.30745657973081, 12.877890575567504, 10.412786184711006, 11.271570037941219, 9.965145615014407, 9.481544958938311, 11.375560858005802, 13.352894587199986]
E[nytimes] = 11.9431

shakespeare
[62.61495266291692, 9.539922101229719, 8.683988364134656, 8.039118824728014, 8.637012861733446, 7.1194526280712305, 12.314782258987226, 10.020211191845359, 9.049313888904358, 7.882180303190417, 9.5210307896922, 8.326431700360278, 11.307418292211821, 9.952907330527133, 7.1055443014494335, 68.91495969355348, 8.630829993561559, 9.68519910718141, 10.115099438925938, 9.842749512110565, 7.943320881867454, 7.720228748452147, 8.959284508193031, 9.534739492966517, 9.259818639128433, 7.9